In [1]:
%load_ext blackcellmagic 
# %%black -l 120
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
from itertools import zip_longest
import numpy as np
import matplotlib.pyplot as plt
from experiments.base.compute_iqm import get_iqm_and_conf_per_epoch
from experiments.atari.exp_output import scale_score


games = ["BattleZone", "DoubleDunk", "NameThisGame"]
experiment_folders = {
    "activation": [
        "activation_tanh_relu_sigmoid_*/adadqnstatic",
        "lr_5e-5_*/dqn",
        "activation_tanh_*/dqn",
        "activation_sigmoid_*/dqn",
    ],
    "architecture": [
        "architecture_44_52_64_512__32_64_64_512__46_46_46_714_*/adadqnstatic",
        "lr_5e-5_*/dqn",
        "architecture_44_52_64_512_*/dqn",
        "architecture_46_46_46_714_*/dqn",
    ],
    "epsilon_adam": [
        "epsilon_adam_small_medium_large_*/adadqnstatic",
        "lr_5e-5_*/dqn",
        "epsilon_adam_medium_eps_*/dqn",
        "epsilon_adam_small_eps_*/dqn",
    ],
}

experiment_data = {
    experiment_type: {
        experiment: {game: {} for game in games + ["aggregate"]} for experiment in experiment_folders[experiment_type]
    }
    for experiment_type in experiment_folders
}

base_path = os.path.join(os.path.abspath(""), "static_hp_set")

def bincount_atleast3D(array):
    bincount = np.bincount(array)
    return np.append(bincount, np.zeros(3 - len(bincount)))

for experiment_type, list_experiments in experiment_folders.items():
    for experiment in list_experiments:
        raw_values = []
        raw_idx_count = []
        for game in games:
            experiment_path = os.path.join(base_path, experiment.replace("*", game), "episode_returns_and_lenghts")

            returns_experiment_ = []
            for experiment_file in os.listdir(experiment_path):
                list_episode_returns = json.load(open(os.path.join(experiment_path, experiment_file), "r"))[
                    "episode_returns"
                ]
                returns_experiment_.append(
                    [scale_score(np.mean(episode_returns), game) for episode_returns in list_episode_returns]
                )

            returns_experiment = np.array(list(zip_longest(*returns_experiment_, fillvalue=np.nan))).T

            print(f"Plot {experiment.replace('*', game)} with {returns_experiment.shape[0]} seeds.")
            n_epochs = json.load(open(os.path.join(experiment_path, "..", "..", "parameters.json"), "r"))["n_epochs"]
            if returns_experiment.shape[1] != n_epochs:
                print(f"!!! None of the seeds are not complete !!!")
            elif np.isnan(returns_experiment).any():
                seeds = np.array(list(map(lambda path: int(path.strip(".json")), os.listdir(experiment_path))))
                print(f"!!! Seeds {seeds[np.isnan(returns_experiment).any(axis=1)]} are not complete !!!")

            raw_values.append(returns_experiment)
            (
                experiment_data[experiment_type][experiment][game]["iqm"],
                experiment_data[experiment_type][experiment][game]["confidence"],
            ) = get_iqm_and_conf_per_epoch(returns_experiment)
            experiment_data[experiment_type][experiment][game]["x_values"] = np.arange(
                1, returns_experiment.shape[1] + 1
            )

            experiment_path_indices = os.path.join(
                base_path, experiment.replace("*", game), "indices_and_hyperparameters_details"
            )

            if os.path.exists(experiment_path_indices):
                indices_experiment_ = []
                for experiment_file in os.listdir(experiment_path_indices):
                    indices_experiment_.append(
                        json.load(open(os.path.join(experiment_path_indices, experiment_file), "r"))[
                                "compute_target"
                            ]
                    )
                nan_filled_indices_ = np.array(list(zip_longest(*indices_experiment_, fillvalue=np.nan))).T.astype(int)
                # split the indices in 40 epochs
                splitted_indices = nan_filled_indices_[:, : -(nan_filled_indices_.shape[1] % 40)].reshape(
                    (nan_filled_indices_.shape[0], 40, nan_filled_indices_.shape[1] // 40)
                )
                # count the number of occurances of each index
                indices_occurances = np.apply_along_axis(
                    bincount_atleast3D, 1, splitted_indices.transpose((1, 0, 2)).reshape((40, -1))
                )
                indices_frequency = indices_occurances.T / np.sum(indices_occurances, axis=1)

                raw_idx_count.append(indices_occurances)
                experiment_data[experiment_type][experiment][game]["idx_target"] = indices_frequency

        (
            experiment_data[experiment_type][experiment]["aggregate"]["iqm"],
            experiment_data[experiment_type][experiment]["aggregate"]["confidence"],
        ) = get_iqm_and_conf_per_epoch(np.array(raw_values))
        experiment_data[experiment_type][experiment]["aggregate"]["x_values"] = np.arange(1, raw_values[0].shape[1] + 1)
        if len(raw_idx_count) > 0:
            idx_count = np.array(raw_idx_count).sum(axis=0)
            experiment_data[experiment_type][experiment]["aggregate"]["idx_target"] = idx_count.T / np.sum(
            	idx_count, axis=1
            )

In [ ]:
from matplotlib.lines import Line2D
from experiments.atari.exp_output import NAMES, COLORS, STYLES, ORDERS


plt.rc("font", family="serif", serif="Times New Roman", size=22)
plt.rc("lines", linewidth=3)

fig, axes = plt.subplots(9, 3, figsize=(17, 10), height_ratios=[1.7, 2, 1, 1.7, 2, 1, 1.7, 2, 1])

for idx_y_axis, experiment_type in enumerate(experiment_data):
    for experiment in experiment_data[experiment_type]:
        for idx_x_axis, game in enumerate(games):
            axes[3 * idx_y_axis, idx_x_axis].axison = False
            hyperparameter, algorithm = experiment.split("_*/")

            ax = axes[3 * idx_y_axis + 1, idx_x_axis]
            ax.plot(
                experiment_data[experiment_type][experiment][game]["x_values"],
                experiment_data[experiment_type][experiment][game]["iqm"],
                color=COLORS[experiment_type][experiment],
                linestyle=STYLES[experiment_type][experiment],
                zorder=ORDERS[experiment_type][experiment],
            )
            ax.fill_between(
                experiment_data[experiment_type][experiment][game]["x_values"],
                experiment_data[experiment_type][experiment][game]["confidence"][0],
                experiment_data[experiment_type][experiment][game]["confidence"][1],
                color=COLORS[experiment_type][experiment],
                alpha=0.3,
                zorder=ORDERS[experiment_type][experiment],
            )

            ax.set_title(game)
            ax.grid(zorder=0)
            ax.set_xlim(1, 40)
            ax.set_xticks([1] + list(range(0, 41, 10))[1:])
            ax.set_xticklabels([])

_ = axes[4, 0].set_ylabel("IQM Human Norm Score", labelpad=10)

In [ ]:
# Compute the ordering of the HPs in final performances
sorted_experiment = {
    experiment_type: {game: {"idx": [], "name": []} for game in games} for experiment_type in experiment_data
}
for experiment_type in experiment_data:
    for game in games:
        final_performances = []
        experiment_names = []
        for experiment in experiment_data[experiment_type]:
            if experiment.split("/")[-1] == "adadqnstatic":
            	continue
            final_performances.append(experiment_data[experiment_type][experiment][game]["iqm"][-1])
            experiment_names.append(experiment)
        sorted_experiment[experiment_type][game]["index"] = np.argsort(final_performances)
        sorted_experiment[experiment_type][game]["name"] = np.array(experiment_names)[np.argsort(final_performances)]

for idx_y_axis, experiment_type in enumerate(experiment_data):
    for experiment in experiment_data[experiment_type]:
        if experiment.split("/")[-1] != "adadqnstatic":
            continue
        hyperparameter, algorithm = experiment.split("_*/")
        for idx_x_axis, game in enumerate(games):
            ax = axes[3 * idx_y_axis + 2, idx_x_axis]
            indices_count = experiment_data[experiment_type][experiment][game]["idx_target"]
            # Target - Over seeds
            bottom = np.zeros(40, dtype=float)

            for name, idx_count in zip(
                sorted_experiment[experiment_type][game]["name"],
                indices_count[sorted_experiment[experiment_type][game]["index"]],
            ):
                ax.bar(
                    experiment_data[experiment_type][experiment][game]["x_values"],
                    idx_count,
                    width=1,
                    bottom=bottom,
                    color=COLORS[experiment_type][name],
                    edgecolor="black",
                )
                bottom += idx_count

            ax.grid(zorder=0)
            ax.set_xticks([1] + list(range(0, 41, 10))[1:])
            ax.set_xlim(1, 40)
            if idx_y_axis == 2:
                ax.set_xlabel("Num Frames (in millions)")
            else:
                ax.set_xticklabels([])
            ax.set_ylim((0, 1))
            ax.set_yticks([])
            ax.set_yticklabels([])

            if idx_x_axis == 0:
                ax.set_ylabel("Target ", labelpad=5, fontsize=18)
fig

In [ ]:
from matplotlib.container import BarContainer
from matplotlib.patches import Rectangle


for idx_y_axis, experiment_type in enumerate(experiment_data):
    axes[3 * idx_y_axis, 1].text(
        0.5, 0.85, NAMES[experiment_type]["name"], horizontalalignment="center", verticalalignment="center", fontsize=25
    )
    
    experiments = list(experiment_data[experiment_type].keys())
    selected_lines = [
        Line2D(
            [], [], color=COLORS[experiment_type][experiments[0]], linestyle=STYLES[experiment_type][experiments[0]]
        ),
        Line2D(
            [], [], color=COLORS[experiment_type][experiments[1]], linestyle=STYLES[experiment_type][experiments[1]]
        ),
        BarContainer(
            [
                Rectangle(
                    (0, 0),
                    0,
                    0,
                    facecolor=COLORS[experiment_type][experiments[1]],
                    edgecolor="black",
                )
            ]
        ),
        Line2D(
            [], [], color=COLORS[experiment_type][experiments[2]], linestyle=STYLES[experiment_type][experiments[2]]
        ),
        BarContainer(
            [
                Rectangle(
                    (0, 0),
                    0,
                    0,
                    facecolor=COLORS[experiment_type][experiments[2]],
                    edgecolor="black",
                )
            ]
        ),
        Line2D(
            [], [], color=COLORS[experiment_type][experiments[3]], linestyle=STYLES[experiment_type][experiments[3]]
        ),
        BarContainer(
            [
                Rectangle(
                    (0, 0),
                    0,
                    0,
                    facecolor=COLORS[experiment_type][experiments[3]],
                    edgecolor="black",
                )
            ]
        ),
    ]
    selected_legends = [
        "  " + NAMES[experiment_type][f"{experiments[0]}"],
        "",
        NAMES[experiment_type][f"{experiments[1]}"],
        "",
        NAMES[experiment_type][f"{experiments[2]}"],
        "",
        NAMES[experiment_type][f"{experiments[3]}"],
    ]

    legend = axes[3 * idx_y_axis + 1, 1].legend(
        selected_lines,
        selected_legends,
        ncols=7,
        frameon=False,
        loc="center",
        bbox_to_anchor=(0.5, 1.56),
        columnspacing=0.5,
        handlelength=1,
        handletextpad=-0.1,
        fontsize=20
    )
    legend.legend_handles[2]._width = 7
    legend.legend_handles[4]._width = 7
    legend.legend_handles[6]._width = 7
    axes[3 * idx_y_axis + 1, 1].legends = [legend]

fig.subplots_adjust(wspace=0.2, hspace=0.25)
fig.savefig(f"{base_path}/performance_per_game.pdf", bbox_inches="tight")
fig

In [ ]:
from matplotlib.lines import Line2D
from experiments.atari.exp_output import NAMES, COLORS, STYLES, ORDERS


plt.rc("font", family="serif", serif="Times New Roman", size=22)
plt.rc("lines", linewidth=3)

fig, axes = plt.subplots(2, 3, figsize=(17, 2.5), height_ratios=[1, 0.5])

for idx_x_axis, experiment_type in enumerate(experiment_data):
    for experiment in experiment_data[experiment_type]:
        hyperparameter, algorithm = experiment.split("_*/")

        ax = axes[0, idx_x_axis]
        ax.plot(
            experiment_data[experiment_type][experiment]["aggregate"]["x_values"],
            experiment_data[experiment_type][experiment]["aggregate"]["iqm"],
            color=COLORS[experiment_type][experiment],
            linestyle=STYLES[experiment_type][experiment],
            zorder=ORDERS[experiment_type][experiment],
        )
        ax.fill_between(
            experiment_data[experiment_type][experiment]["aggregate"]["x_values"],
            experiment_data[experiment_type][experiment]["aggregate"]["confidence"][0],
            experiment_data[experiment_type][experiment]["aggregate"]["confidence"][1],
            color=COLORS[experiment_type][experiment],
            alpha=0.3,
            zorder=ORDERS[experiment_type][experiment],
        )

        ax.grid(zorder=0)
        ax.set_xlim(1, 40)
        ax.set_xticks([1] + list(range(0, 41, 10))[1:])
        ax.set_xticklabels([])

_ = axes[0, 0].set_ylabel("IQM Human Norm Score")

In [ ]:
# Compute the ordering of the HPs in final performances
sorted_experiment = {experiment_type: {"idx": [], "name": []} for experiment_type in experiment_data}
for experiment_type in experiment_data:
    final_performances = []
    experiment_names = []
    for experiment in experiment_data[experiment_type]:
        if experiment.split("/")[-1] == "adadqnstatic":
            continue
        final_performances.append(experiment_data[experiment_type][experiment]["aggregate"]["iqm"][-1])
        experiment_names.append(experiment)
    sorted_experiment[experiment_type]["index"] = np.argsort(final_performances)
    sorted_experiment[experiment_type]["name"] = np.array(experiment_names)[np.argsort(final_performances)]

for idx_x_axis, experiment_type in enumerate(experiment_data):
    for experiment in experiment_data[experiment_type]:
        if experiment.split("/")[-1] != "adadqnstatic":
            continue
        hyperparameter, algorithm = experiment.split("_*/")

        ax = axes[1, idx_x_axis]
        indices_count = experiment_data[experiment_type][experiment]["aggregate"]["idx_target"]
        # Target - Over seeds
        bottom = np.zeros(40, dtype=float)

        for name, idx_count in zip(
            sorted_experiment[experiment_type]["name"], indices_count[sorted_experiment[experiment_type]["index"]]
        ):
            ax.bar(
                experiment_data[experiment_type][experiment]["aggregate"]["x_values"],
                idx_count,
                width=1,
                bottom=bottom,
                color=COLORS[experiment_type][name],
                edgecolor="black",
            )
            bottom += idx_count

        ax.grid(zorder=0)
        ax.set_xticks([1] + list(range(0, 41, 10))[1:])
        ax.set_xlim(1, 40)
        ax.set_xlabel("Num Frames (in millions)")
        ax.set_ylim((0, 1))
        ax.set_yticks([])
        ax.set_yticklabels([])

        if idx_x_axis == 0:
            ax.set_ylabel("Target", labelpad=5)

_ = axes[0, 0].set_ylabel("IQM Human Norm Score")
fig

In [ ]:
from matplotlib.container import BarContainer
from matplotlib.patches import Rectangle


for idx_x_axis, experiment_type in enumerate(experiment_data):
    experiments = list(experiment_data[experiment_type].keys())
    selected_lines = [
        Line2D(
            [], [], color=COLORS[experiment_type][experiments[0]], linestyle=STYLES[experiment_type][experiments[0]]
        ),
        Line2D(
            [], [], color=COLORS[experiment_type][experiments[1]], linestyle=STYLES[experiment_type][experiments[1]]
        ),
        BarContainer([Rectangle((0, 0), 0, 0, facecolor="white")]),
        BarContainer(
            [
                Rectangle(
                    (0, 0),
                    0,
                    0,
                    facecolor=COLORS[experiment_type][experiments[1]],
                    edgecolor="black",
                )
            ]
        ),
        Line2D(
            [], [], color=COLORS[experiment_type][experiments[2]], linestyle=STYLES[experiment_type][experiments[2]]
        ),
        Line2D(
            [], [], color=COLORS[experiment_type][experiments[3]], linestyle=STYLES[experiment_type][experiments[3]]
        ),
        BarContainer(
            [
                Rectangle(
                    (0, 0),
                    0,
                    0,
                    facecolor=COLORS[experiment_type][experiments[2]],
                    edgecolor="black",
                )
            ]
        ),
        BarContainer(
            [
                Rectangle(
                    (0, 0),
                    0,
                    0,
                    facecolor=COLORS[experiment_type][experiments[3]],
                    edgecolor="black",
                )
            ]
        ),
    ]
    selected_legends = [
        "  ",
        "  ",
        NAMES[experiment_type][f"{experiments[0]}"],
        NAMES[experiment_type][f"{experiments[1]}"],
        "  ",
        "  ",
        NAMES[experiment_type][f"{experiments[2]}"],
        NAMES[experiment_type][f"{experiments[3]}"],
    ]

    legend = axes[0, idx_x_axis].legend(
        selected_lines,
        selected_legends,
        ncols=4,
        frameon=False,
        loc="center",
        bbox_to_anchor=(0.5, 1.28),
        columnspacing=0.5,
        fontsize=15,
        handlelength=1.5,
        handletextpad=-0.6,
    )
    legend.legend_handles[2]._width = 7
    legend.legend_handles[3]._width = 7
    legend.legend_handles[6]._width = 7
    legend.legend_handles[7]._width = 7
    axes[0, idx_x_axis].set_title(NAMES[experiment_type]["short_name"], pad=52)

fig.subplots_adjust(wspace=0.1, hspace=0.17)
fig.savefig(f"{base_path}/performance.pdf", bbox_inches="tight")
fig